In [11]:
import requests
import pandas as pd
import warnings
# Hide FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# URL for Leeds 
url = "https://fbref.com/en/comps/12/stats/players/La-Liga-Stats"

# Sendeing HTTP GET-request to webpage
data = requests.get(url)

response = data.status_code

# check for success response (statuscode 200)
if response == 200:
    print("Successfully retrieved page")
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")

Successfully retrieved page


In [1]:
from bs4 import BeautifulSoup
import pandas as pd
with open("../../html.txt", "r", encoding="utf-8") as file:
    html_content = file.read()
# Create a BeautifulSoup-objekt for navigating in HTML
soup = BeautifulSoup(html_content, "lxml")

# Print first 500 characters to get an overview
print(soup.prettify()[:500]) 

<!DOCTYPE html>
<html class="no-js" data-root="/home/fb/deploy/www" data-version="klecko-" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
  <link href="https://cdn.ssref.net/req/202508071" rel="dns-prefetch"/>
  <script>
   /* https://docs.osano.com/hc/en-us/articles/22469433444372-Google-Consent-Mode-v2  */
  window.dataLayer = window.dataLayer ||[]


In [58]:
# finding all tables in page
tables = soup.find_all("table") # finding table-elements
# print all ids for each table
for table in tables:
    print(table.get("id")) #print all ids

stats_squads_standard_for
stats_squads_standard_against
stats_standard


In [3]:
# table= soup.select("#stats_standard") #using '#' to indicate id we want 
table= tables[2] #soup creates lists, so get first element
df=pd.read_html(str(table))[0]
df.head()


/tmp/ipykernel_3111/3219594612.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(table))[0]


Unnamed: 0_level_0   Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk               Player             Nation   
0                  1  Jones El-Abdellaoui             ma MAR   
1                  2          Abdel Abqar             ma MAR   
2                  3           Akor Adams             ng NGA   
3                  4    David Affengruber             at AUT   
4                  5   Julen Agirrezabala             es ESP   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0  \
                 Pos              Squad                Age               Born   
0              MF,DF         Celta Vigo             19-256               2006   
1                 DF             Getafe             26-199               1999   
2                 FW            Sevilla             25-239               2000   
3                 DF              Elche             24-190               2001   
4                 GK           Valencia             24-273               2000   

  Playing Time              ... Per 90 Minutes                                 \
            MP Starts  Min  ...            Ast   G+A  G-PK G+A-PK    xG   xAG   
0            1      0   34  ...           0.00  0.00  0.00   0.00  0.00  0.14   
1            3      3  252  ...           0.00  0.00  0.00   0.00  0.02  0.00   
2            4      3  286  ...           0.31  0.31  0.00   0.31  0.31  0.03   
3            5      5  450  ...           0.00  0.00  0.00   0.00  0.00  0.02   
4            6      6  540  ...           0.00  0.00  0.00   0.00  0.00  0.00   

                        Unnamed: 36_level_0  
  xG+xAG  npxG npxG+xAG             Matches  
0   0.14  0.00     0.14             Matches  
1   0.02  0.02     0.02             Matches  
2   0.34  0.31     0.34             Matches  
3   0.02  0.00     0.02             Matches  
4   0.00  0.00     0.00             Matches  

[5 rows x 37 columns]

In [4]:
rows = []
for tr in table.find_all("tr"):
    row = []
    for td in tr.find_all(["td", "th"]):
        link = td.find("a")
        if link:
            # Guardar tanto el texto como el link
            row.append((link.text.strip(), link["href"]))
        else:
            row.append(td.get_text(strip=True))
    rows.append(row)

In [54]:
df = pd.DataFrame(rows)
# print(df.head())

In [55]:
# print(df.head()) 

df = df.rename(columns = {0: "Rk", 1: "Player", 2: "Nation", 3: "Pos", 4: "Squad", 5: "Age", 6: "Born", 7: "MP", 8: "Starts", 9: "Min", 10: "90s", 11: "Gls", 12: "Ast", 13: "G+A", 14: "G-PK", 15: "PK", 16: "PKatt", 17: "CrdY", 18: "CrdR", 19: "xG", 20: "npxG", 21: "xAG", 22: "npxG+xAG", 23: "PrgC", 24: "PrgP", 25: "PrgR", 26:"Gls-90min", 27:"Ast-90min", 28:"G+A-90min", 29:"G-PK-90min", 30:"G+A-PK-90min", 31:"xG-90min", 32:"xAG-90min", 33:"xG+xA-90min", 34:"npxG-90min", 35:"npxG+xAG-90min", 36:"Matches"})
                                                                                          
# print(df[1])
# print(df.iloc[3])

In [56]:
#preprocess data
df.Player = df.Player.apply(lambda x: x[0] if isinstance(x, tuple) else x)
df.Nation = df.Nation.apply(lambda x: x[0] if isinstance(x, tuple) else x)
df.Squad = df.Squad.apply(lambda x: x[0] if isinstance(x, tuple) else x)
df.Matches = df.Matches.apply(lambda x: x[1] if isinstance(x, tuple) else x)

df = df.drop([0, 1])
df = df.reset_index(drop=True)


In [57]:
print(df.head())

  Rk               Player  Nation    Pos       Squad     Age  Born MP Starts  \
0  1  Jones El-Abdellaoui  ma MAR  MF,DF  Celta Vigo  19-256  2006  1      0   
1  2          Abdel Abqar  ma MAR     DF      Getafe  26-199  1999  3      3   
2  3           Akor Adams  ng NGA     FW     Sevilla  25-239  2000  4      3   
3  4    David Affengruber  at AUT     DF       Elche  24-190  2001  5      5   
4  5   Julen Agirrezabala  es ESP     GK    Valencia  24-273  2000  6      6   

   Min  ... Ast-90min G+A-90min G-PK-90min G+A-PK-90min xG-90min xAG-90min  \
0   34  ...      0.00      0.00       0.00         0.00     0.00      0.14   
1  252  ...      0.00      0.00       0.00         0.00     0.02      0.00   
2  286  ...      0.31      0.31       0.00         0.31     0.31      0.03   
3  450  ...      0.00      0.00       0.00         0.00     0.00      0.02   
4  540  ...      0.00      0.00       0.00         0.00     0.00      0.00   

  xG+xA-90min npxG-90min npxG+xAG-90min  \
0      

In [28]:
import os
import pandas as pd
import requests
import time
import random
from tqdm import tqdm  # para barra de progreso

def player_matches_scrap(players_df, save_path='data/raw/fbref_player_matches.parquet'):
    """
    Scrap de partidos de jugadores de FBref y guarda un DataFrame consolidado en Parquet.
    Muestra barra de progreso con tiempo estimado restante.
    """
    all_player_matches = []
    headers = {"User-Agent": "Mozilla/5.0"}
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    total_players = len(players_df)
    start_time = time.time()

    # tqdm para barra de progreso
    for i, row in enumerate(tqdm(players_df.itertuples(index=False), total=total_players, desc="Scraping jugadores")):
        # print(i)
        # if i==2:
        #     break
        player_name = getattr(row, "Player")
        url = getattr(row, "Matches")

        if pd.isna(url):
            continue

        full_url = f"https://fbref.com{url}"
        response = requests.get(full_url, headers=headers)
        if response.status_code != 200:
            print(f"⚠️ No se pudo obtener {player_name}, respuesta {response.status_code}")
            continue

        tables = pd.read_html(response.text)
        if len(tables) == 0:
            continue

        df_matches = tables[0]

        # 🔹 Aplanar columnas MultiIndex y limpiar nombres
        if isinstance(df_matches.columns, pd.MultiIndex):
            new_cols = []
            for col in df_matches.columns:
                if col[0].startswith("Unnamed"):
                    new_cols.append(col[1])
                else:
                    new_cols.append(f"{col[0]}_{col[1]}" if col[1] else col[0])
            df_matches.columns = new_cols

        # 🔹 Añadir contexto del jugador
        df_matches["Player"] = player_name
        df_matches["PlayerLink"] = url

        all_player_matches.append(df_matches)

        # 🔹 Retraso aleatorio
        time.sleep(random.uniform(3, 8))

        # 🔹 Actualizar tiempo estimado restante en tqdm
        elapsed = time.time() - start_time
        avg_time_per_player = elapsed / (i + 1)
        remaining_time_sec = avg_time_per_player * (total_players - (i + 1))
        mins, secs = divmod(int(remaining_time_sec), 60)
        tqdm.write(f"⏳ Tiempo estimado restante: {mins}m {secs}s")

    # 🔹 Concatenar todo
    if all_player_matches:
        big_df = pd.concat(all_player_matches, ignore_index=True)

        # 🔹 Conversión segura de tipos para Parquet
        for col in big_df.columns:
            try:
                big_df[col] = pd.to_numeric(big_df[col])
            except:
                big_df[col] = big_df[col].astype(str)

        big_df.to_parquet(save_path, index=False)
        print(f"✔ Guardada tabla consolidada en {save_path}")
        return big_df
    else:
        print("⚠️ No se pudo obtener ningún partido")
        return pd.DataFrame()


In [ ]:
players_stats = pd.read_parquet("../../data/raw/fbref_players_stats.parquet")
tables = player_matches_scrap(players_stats)

player_matches = tables
player_matches.to_parquet("../../data/raw/fbref_player_matches.parquet", index=False)

Scraping jugadores:   0%|          | 0/479 [00:00<?, ?it/s]

/tmp/ipykernel_2900/160472000.py:37: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)
Scraping jugadores:   0%|          | 1/479 [00:04<34:59,  4.39s/it]/tmp/ipykernel_2900/160472000.py:37: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


⏳ Tiempo estimado restante: 35m 1s


Scraping jugadores:   0%|          | 2/479 [00:09<37:58,  4.78s/it]/tmp/ipykernel_2900/160472000.py:37: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


⏳ Tiempo estimado restante: 37m 31s


...

⏳ Tiempo estimado restante: 43m 42s


Scraping jugadores:   5%|▍         | 22/479 [02:05<41:12,  5.41s/it]/tmp/ipykernel_2900/160472000.py:37: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


⏳ Tiempo estimado restante: 43m 28s


Scraping jugadores:   5%|▍         | 23/479 [02:08<35:53,  4.72s/it]/tmp/ipykernel_2900/160472000.py:37: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


⏳ Tiempo estimado restante: 42m 31s


In [24]:
print(player_matches.head())

         Date  Day     Comp        Round Venue Result       Squad    Opponent  \
0  2025-08-17  Sun  La Liga  Matchweek 1  Home  L 0–2  Celta Vigo      Getafe   
1  2025-08-23  Sat  La Liga  Matchweek 2  Away  D 1–1  Celta Vigo    Mallorca   
2  2025-08-27  Wed  La Liga  Matchweek 6  Home  D 1–1  Celta Vigo       Betis   
3  2025-08-31  Sun  La Liga  Matchweek 3  Home  D 1–1  Celta Vigo  Villarreal   
4  2025-09-14  Sun  La Liga  Matchweek 4  Home  D 1–1  Celta Vigo      Girona   

  Start                                  Pos  ...  \
0     N                                RM,RB  ...   
1     N  On matchday squad, but did not play  ...   
2     N  On matchday squad, but did not play  ...   
3     N  On matchday squad, but did not play  ...   
4     N  On matchday squad, but did not play  ...   

                            Passes_Att                          Passes_Cmp%  \
0                                   26                                 80.8   
1  On matchday squad, but did not pl